Proyecto 2. Introducción al análisis de datos
Elsa Ximena Barba Flores

In [1]:
# Comenzamos importando la librería Pandas para análisis de datos y Seaborn para graficar
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Como una buena práctica se define una variable con el nombre del archivo CSV
synergy_db = "synergy_logistics_database.csv"

In [3]:
# Abrimos el archivo CSV en modo lectura
sldf = pd.read_csv(synergy_db, parse_dates=[5])


In [4]:
# Comenzamos creando un dataframe para las importaciones y otro para las exportaciones
importaciones = sldf[ sldf["direction"] == "Imports"]
exportaciones = sldf[ sldf["direction"] == "Exports"]

Opción 1: Rutas de importación y exportación.

Synergy Logistics quiere saber cuáles son las diez rutas más demandadas. Para resolver esto primero hay que tener en cuénta ¿Qué es lo que consideramos como ruta? Para definir una ruta es necesario tomar en cuenta el origen y el destino (no es lo mismo una ruta de China a Japón que de Japón a China) y el medio de transporte (no es lo mismo ir de México a Estados Unidos por tierra que por mar). Se consideran por separado los productos que son de importación de los de exportación.

In [ ]:
# Utilizamos el método "groupby" para delimitar qué columnas de la variable dataframe queremos reacomodar
# El argumento "by" nos permite delimitar cuáles columnas queremos obtener como índices.
rutas = sldf.groupby(by=['direction', 'origin', 'destination', 'transport_mode'])
#rutas.sum()

In [ ]:
# Sumamos los valores de la columna "total_value"
suma = rutas.sum()['total_value']
# Agregamos estadísticas descriptivas a la columna "total_value" de la variable "rutas"
rutas = rutas['total_value'].describe()
# Añadimos la columna con la suma total
rutas['suma_total'] = suma
rutas = rutas.reset_index()
#rutas

Ahora disponemos de un dataframe que contiene columnas (índices) que nos indican si se trata de una importación o una exportación, el país de origen, destino, medio de transporte, cantidad de veces que se realizó la ruta y la suma total de los valores generados por cada ruta.

Con estos datos, hacemos el análisis para las diez rutas de exportación y las diez rutas de importación más demandadas

In [ ]:
# Creamos dos df separados, uno que contenga solo las importaciones y uno que contenga solamente exportaciones
exportaciones = rutas[rutas['direction'] == 'Exports']
#exportaciones
importaciones = rutas[rutas['direction'] == 'Imports']
#importaciones

In [ ]:
# Obtenemos un número entero que sea la suma de todos los valores de exportación
# Y otro que contenga la suma de todos los valores de importación
valor_total_exp = exportaciones.suma_total.sum()
#valor_total_exp
valor_total_imp = importaciones.suma_total.sum()

In [ ]:
# Creación de una función con argumentos dataframe y un mensaje
def resultados(df, m):
    total_valor = df.suma_total.sum()
    total_usos = df['count'].sum()
    porcentaje = (total_valor / valor_total_exp)*10000
    porcentaje = int(porcentaje) / 100
    print(f'Las 10 rutas {m} aportan {porcentaje}%, en un total de: {total_usos} servicios')

Mostrando las diez rutas de exportación más demandadas

In [ ]:
mas_demandadas_exp = exportaciones.sort_values(by='count', ascending=False).head(10)
mas_demandadas_exp = mas_demandadas_exp.reset_index()
mayor_valor_exp = exportaciones.sort_values(by='suma_total', ascending=False).head(10)
mayor_valor_exp = mayor_valor_exp.reset_index()
#mas_demandadas_exp
#mayor_valor_exp

In [ ]:
resultados(mas_demandadas_exp, 'mas demandadas')
resultados(mayor_valor_exp, 'con mayor valor')

Hacemos el análisis para las diez rutas de importación más demandadas

In [ ]:
mas_demandadas_imp = importaciones.sort_values(by='count', ascending=False).head(10)
mas_demandadas_imp = mas_demandadas_imp.reset_index()
mayor_valor_imp = importaciones.sort_values(by='suma_total', ascending=False).head(10)
mayor_valor_imp = mayor_valor_imp.reset_index()
#mas_demandadas_imp
#mayor_valor_imp

In [ ]:
resultados(mas_demandadas_imp, 'mas demandadas')
resultados(mayor_valor_imp, 'con mayor valor')

Opción 2: Medio de transporte utilizado

Synergy Logistics espera que realicemos un análisis que muestre los tres medios de transporte más importantes para las rutas de importación y exportación tomando en cuenta los valores que cada una genera.

A primera instancia podemos pre visualizar cuáles son las rutas más importantes haciendo un gráfico de barras que muestre las insidencias. 

In [ ]:
ax = sns.countplot(x='transport_mode', data=sldf, hue="direction")

Del grafico superior es fácil visualizar que, tanto para exportaciones como para importaciones, los medios de transporte más utilizados son por mar, por riel y por carretera.
Ahora sigue hacer un análisis tomando en cuenta los valores de las ganancias, para facilitar la visualización realizamos un análisis anual.

In [ ]:
#Obtenemos un dataframe agrupando por año y medio de transporte
transportes_anuales = sldf.groupby(by=['year', 'transport_mode'])
# Creamos una lista en la que tengamos como índices los años y los medios de transporte
#además sumamos los valores obtenidos para cada uno de ellos
valor_anual_transporte = transportes_anuales['total_value'].agg(pd.Series.sum)
valor_anual_transporte

In [ ]:
# Creamos una nueva variable dataframe
info_transp_anual = pd.DataFrame()
# Guardamos la serie "valor_anual_transporte" en el dataframe
info_transp_anual['valor_total'] = valor_anual_transporte
# Añadimos una columna de frecuencia que cuenta la cantidad de incidencias
info_transp_anual['frecuencia'] = transportes_anuales['total_value'].describe()['count']
info_transp_anual

Ahora mostramos un gráfico de los años contra la frecuencia con la que se se utilizaron los distintos transportes contra el paso de los años.

In [ ]:
sns.lineplot(x='year', y='frecuencia', hue='transport_mode', data=info_transp_anual)

A continuación se grafica el valor de las importaciones y exportaciones realizadas contra el paso de los años.

In [ ]:
sns.lineplot(x='year', y='valor_total', hue='transport_mode', data=info_transp_anual)

Opción 3: Valor total de importaciones y exportaciones

Synergy Logistics desea enfocarse en los países que generan el 80% de sus ganancias en importaciones y exportaciones.


In [ ]:
impo_origen = importaciones.groupby("origin").sum()["total_value"].sort_values(ascending=False)
impo_destino = importaciones.groupby("destination").sum()["total_value"].sort_values(ascending=False)

expo_origen = exportaciones.groupby("origin").sum()["total_value"].sort_values(ascending=False)
expo_destino = exportaciones.groupby("destination").sum()["total_value"].sort_values(ascending=False)